In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

from src.analyze_band import analyze_band


# audio_path = "../samples/sample8-ielts8.5.wav"
audio_path = "../samples/sample7.flac"
# audio_path = "../data/l2arctic_spontaneous/L2A_001.wav"


final_result = analyze_band(audio_path)
final_result

d:\Work\Projects\AI\speech-analysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Analyzing audio: ../samples/sample7.flac
Context: conversational

[1/5] Transcribing with Whisper (verbatim)...
Prosody Variation (F0 std): 41.33 Hz
Monotone Detected: False
  Duration: 31.28s
  Words: 83

[2/5] Marking filler words...
  Marked: 0 filler words
  Content words: 83

[3/5] Aligning words with WhisperX...
  Aligned: 83 words

[4/5] Detecting subtle fillers with Wav2Vec2...


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  Total events: 3

[5/5] Calculating raw score...


{'band_scores': {'overall_band': 7.5,
  'criterion_bands': {'fluency_coherence': 7.0,
   'lexical_resource': 7.0,
   'grammatical_range_accuracy': 8.0,
   'pronunciation': 9.0},
  'feedback_summary': 'Weakest: Fluency (7.0), Strongest: Pronunciation (9.0)',
  'duration_factor_applied': {'threshold_mult': 0.85, 'penalty_mult': 1.2}},
 'analysis': {'metadata': {'audio_duration_sec': 31.28,
   'speaking_time_sec': np.float64(29.2),
   'total_words_transcribed': 83,
   'content_word_count': 83,
   'analysis_timestamp': '2026-01-16T11:16:13.550389Z'},
  'fluency_coherence': {'pauses': {'pause_frequency_per_min': 9.59079283887468,
    'long_pause_rate': 0.0,
    'pause_variability': 0.0},
   'rate': {'speech_rate_wpm': 159.2071611253197,
    'speech_rate_variability': 0.33858014782095264},
   'disfluency': {'filler_frequency_per_min': 1.150895140664962,
    'stutter_frequency_per_min': 0.0,
    'repetition_rate': 0.08163265306122448},
   'coherence': {'coherence_breaks': 1, 'topic_relevance'